In [82]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pyperclip
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome()
driver.get("https://m.cafe.daum.net/beautiful5060/NhGV?boardType=")

In [75]:
url_list = []

In [64]:
for _ in range(10):
    try:
        for page in range(3, 7): # 1~5 페이지 넘기기
            soup = bs(driver.page_source, 'lxml')

            # 게시글 url 링크 가져오기
            urls = soup.select('a', class_='link_cafe make-list-uri.#article_list')

            for k in urls:
                url_list.append('https://m.cafe.daum.net/' + k['href'])

            # url 링크 정리
            filtered_urls = [url for url in url_list if url.startswith('https://m.cafe.daum.net//beautiful5060/NhGV/') and 'comments' not in url]
            filtered_urls = filtered_urls[6:] 

            # 맨 밑으로 스크롤
            time.sleep(0.5)
            body = driver.find_element(By.CSS_SELECTOR, "body")

            time.sleep(0.5)
            body.send_keys(Keys.ENTER)

            # 페이지 넘기기
            time.sleep(0.5)
            driver.find_element(By.CSS_SELECTOR, '#pagingNav > span:nth-child({}) > a'.format(page)).click()

            time.sleep(0.5)
            soup = bs(driver.page_source, 'lxml')

        # > 버튼 클릭
        time.sleep(1)
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[2]/a[2]').click()

    except:
        break

In [78]:
# 겹치는 url 제거
unique_urls = list(set(filtered_urls))

In [72]:
title_list = []
context_list = []

In [73]:
# url 돌면서 제목, 내용 가져오기
for url in unique_urls:
    driver.get(url)
    time.sleep(0.5)

    soup = bs(driver.page_source, 'lxml')

    # 제목
    title = driver.find_element(By.CLASS_NAME, 'article_title').text
    title_list.append(title)

    # 내용
    context = driver.find_element(By.CLASS_NAME, 'tx-content-container').text
    context_list.append(context)

In [80]:
data = {'제목' : title_list, "내용" : context_list}

In [83]:
# 데이터프레임 생성
df = pd.DataFrame(data)

In [86]:
# csv 파일로 저장
df.to_csv('아름다운 5060_여성 휴게실.csv', encoding='utf-8-sig', index=False)